# Introduccion al dataset

El dataset propuesto propuesto proviene de: https://www.propertypriceregister.ie/ y contiene datos de transacciones inmobiliarias en Irlanda desde 2010 hasta la actualidad.

En Irlanda las zonas estan organizadas en 139 identificadores de Ruta unicos associados al codigo postal llamado EIRCODE.
Los primeros 3 elementos de este codigo son siempre el identificador de ruta y los 4 siguientes representan de manera unica la propiedad.
Por desgracia la base de datos de EIRCODEs es paga ($3000 eu) con lo cual no podre utilizarla en este proceso pero podria ser una herramienta para mejorar la calidad de lo datos de llegar a convertir esto en un producto.

# Objetivo personal y motivacion

Hace unos años mientras vivia en Irl se me ocurrio desarrollar una AI capaz de realizar calculos de rentabilidad economica al momento de realizar una transaccion inmobiliaria. Los posibles usos serian multiples:

1- Entender en que valor deberias posicionar un inmueble que quieres vender.

2- Validar si el precio de un inmuelble esta por debajo, arriba o valor de mercado.

3- Buscar potentenciales inmuebles que tengan un indice de rentabilidad alto.

4- Estipular el valor de un inmueble a futuro

5- Entender la tendencia del mercado inmobiliario (a la suba o baja) para saber si es momento de alquilar, comprar o vender


Este servicio se podria ofrecer al publico directo con una serie de opciones gratuitas y de manera paga (premiun) con informacion mas detallada en diferentes categorias.

# Enfoque de este trabajo

Se busca que los datos contengan 3 campos principales: Valor de la transaccion, Fecha, ID de ruta y tamaño aprox de la propiedad. 

La data esta bastante incompleta, especialemente los primeros años de carga, se buscara incrementar el nivel de completitud de 17% a un 80%.

Para esto se emplearan varios pasos:

1- Normalizacion y limpieza de los campos

2- Conversion de la direccion (Address) a sus componentes Street, Neighbourhood, Area tanto sea posible ya que no tiene una estructura fija

3- Conversion de la calles (Street) a su correspondiente numero (si existe) y calle

4- Enriquesimiento de datos conociendo los codigos de ruta (esto es publico y esta en wikipedia) y sus correspondientes ciudades y pueblos

5- Enriquesimiento de datos antiguos usando las entradas actuales que podeen el ERICODE, llamare a esto reconocimiento de vecinos (neighbour)

Una vez procesada la data se organizaran los elementos por codigo de ruta y se aplicaran diferentes analisis matematicos para calcular los precios historicos y futuros por codigo de ruta.

Se espera que al final una simple funcion dado un EIRCODE y el Tamaño de la propiedad pueda estimar el costo minimo, medio y maximo de una dicha propiedad.

# Primera impresion del dataset

El dataset contiene aprox 725k entradas, el campo fecha y valor esta siempre presente.

El campor address varia bastante en estructura pero tambien es el que mas informacion provee.

El mayor desafio sera competar los codigos de ruta y el tamaño de las propiedades.

Mapa de Rutas de Dublin

![IRL MAP](https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/EircodeDublin.svg/1024px-EircodeDublin.svg.png)

# Imports para todo el proyecto

In [1]:
import numpy as np
import pandas as pd
import zipfile # para descomprimir archivos zip
import urllib.request # para descargar de URL
from os import path as osp
import re
import ssl
ruta = osp.join("res", "PPR-ALL.zip")

# Descarga y carga del dataset

In [2]:

# descargar dataset 
url = 'https://www.propertypriceregister.ie/website/npsra/ppr/npsra-ppr.nsf/Downloads/PPR-ALL.zip/$FILE/PPR-ALL.zip'  
ssl._create_default_https_context = ssl._create_unverified_context

# Descargar usando un contexto SSL sin verificación
with urllib.request.urlopen(url, context=ssl._create_unverified_context()) as response, open(ruta, 'wb') as out_file:
	out_file.write(response.read())


In [ ]:
# Unzip file
ruta_ext = osp.join("res")
with zipfile.ZipFile(ruta, 'r') as z: 
    print('Extracting all files...') 
    z.extractall(ruta_ext)
print('Done!')

Extracting all files...
Done!


In [109]:
# Load dataset
ruta_csv = osp.join("res", "PPR-ALL.csv")
properties_dataset = pd.read_csv(ruta_csv, sep=',', header=0, engine='python', encoding='latin1')


In [110]:
# Load Eircode CSV -- This was obtained from wikipedia
eircodes_ruta = osp.join("res", "eircodes.csv")
eircodes_csv = pd.read_csv(eircodes_ruta, sep=',', header=0, engine='python')

eircodes_csv['PostTown'] = eircodes_csv['PostTown'].str.strip().str.lower()
eircodes_csv['PostCounty'] = eircodes_csv['PostCounty'].str.strip().str.lower()
eircodes_csv['RoutingKey'] = eircodes_csv['RoutingKey'].str.strip().str.lower()

# Normalizacion y limpieda de los campos

In [111]:
# Process dataset

properties_dataset = properties_dataset.drop('Not Full Market Price', axis=1)
properties_dataset = properties_dataset.drop('VAT Exclusive', axis=1)

# Format Date -  Date of Sale (dd/mm/yyyy)
properties_dataset = properties_dataset.rename(columns={'Date of Sale (dd/mm/yyyy)': 'Date'})
properties_dataset['Date'] = pd.to_datetime(properties_dataset['Date'], format='%d/%m/%Y', errors='coerce')

# Format Price -Price ()
properties_dataset = properties_dataset.rename(columns={'Price ()': 'Price'})
#properties_dataset.columns.values[4] = 'Price'  # Renaming the column to 'Price' Every time I open the file the char changes...
properties_dataset['Price'] = properties_dataset['Price'].str.replace(r'[^0-9\.\-]', '', regex=True)
properties_dataset['Price'] = pd.to_numeric(properties_dataset['Price'], errors='coerce')

# Format Address - Address
properties_dataset['Address'] = properties_dataset['Address'].str.strip().str.lower()

# Format County - County
properties_dataset['County'] = properties_dataset['County'].str.strip().str.lower()

# Format Eircode - Eircode
properties_dataset['Eircode'] = properties_dataset['Eircode'].str.strip().str.lower()

# Format Description - Description of Property
properties_dataset = properties_dataset.rename(columns={'Description of Property': 'Description'})
properties_dataset['Description'] = properties_dataset['Description'].str.strip().str.lower()

# Format Size - Property Size Description
properties_dataset = properties_dataset.rename(columns={'Property Size Description': 'Size'})
properties_dataset['Size'] = properties_dataset['Size'].str.strip().str.lower()


# Set types
properties_dataset['Address'] = properties_dataset['Address'].astype('string')
properties_dataset['County'] = properties_dataset['County'].astype('string')
properties_dataset['Eircode'] = properties_dataset['Eircode'].astype('string')
properties_dataset['Description'] = properties_dataset['Description'].astype('string') # This will be category later
properties_dataset['Size'] = properties_dataset['Size'].astype('string') # This will be category later





# Conversion de la direccion (Address, Street, Area)

In [112]:
# Split the address into components # TODO: Some Streets have "number, street name" (e.g., "47, Main Street, Dublin") but its very rare
properties_dataset[['Street', 'Neighbourhood', 'Area']] = properties_dataset['Address'].str.split(',', expand=True)
properties_dataset['Street'] = properties_dataset['Street'].astype('string').str.strip()
properties_dataset['Neighbourhood'] = properties_dataset['Neighbourhood'].astype('string').str.strip()
properties_dataset['Area'] = properties_dataset['Area'].astype('string').str.strip()

In [113]:
# Function to format street names
def format_street_name(street):
    if isinstance(street, str):
        street = street.strip()
        elements = street.split(' ')
        if (len(elements) == 0):
            return ''
        elif (elements[0].isdigit()):
            return street # The street starts with a number, return as is
        elif (len(elements) == 1):
            return "0 " + street # Add a leading zero if no number is present and has a single element
        else: # The street does not start with a number, it might have a number in the middle or end
            for i in range(len(elements)):
                potential_number = re.sub(r'[^0-9]', '', elements[i])
                if potential_number.isdigit():
                    number_part = potential_number
                    elements.remove(elements[i])  # Remove the number part from the list
                    elements = [el for el in elements if el not in ['apt', 'apt.','and', 'flat', 'no.', 'site', 'apartment']]
                    text_part = ' '.join(elements)
                    return number_part + " " + text_part.strip()
            text_part = ' '.join(elements) # Street does not contain a number, return with a leading zero
            number_part = "0"
            return number_part + " " + text_part.strip()
    return street

# Apply the function to format street names and split into SNumber and SRoad
properties_dataset['Street'] = properties_dataset['Street'].apply(format_street_name)
properties_dataset[['SNumber', 'SRoad']] = properties_dataset['Street'].str.split(' ', n=1, expand=True)
properties_dataset['SRoad'] = properties_dataset['SRoad'].astype('string').str.strip()

In [114]:
# Function to format area
def format_area(area):
    if isinstance(area, str):
        area = area.strip()
        area = area.replace('.', ' ')  # Replace dots with spaces
        elements = area.split(' ')
        # Remove unwanted elements
        elements = [el for el in elements if el not in ['co', 'co.', 'county', 'road', 'st', 'rd', 'upper', 'lower', 'w', 'e', 'n', 's', 'fw']]
        if len(elements) > 2:
            if(elements[-1].isdigit() and elements[-2] == 'dublin'):
                # If the last two elements are dublin followed by a number, keep only them
                elements = elements[-2:] 
        if len(elements) == 0:
            return ''
        return ' '.join(elements).strip()
    return area

properties_dataset['Area'] = properties_dataset['Area'].apply(format_area)


# Conversion del EIRCODE en Ruta y UID

In [115]:

# To split a string on the 3rd character in a pandas column, use .str.slice
# Slice only if the Eircode is not nan or empty
mask = properties_dataset['Eircode'].notna() & (properties_dataset['Eircode'] != '') & (properties_dataset['Eircode'] != 'nan')
properties_dataset['EIRRouteKey'] = np.where(mask, properties_dataset['Eircode'].str.slice(0, 3), np.nan)
properties_dataset['EIRUID'] = np.where(mask, properties_dataset['Eircode'].str.slice(3), np.nan)

properties_dataset['EIRRouteKey'] = properties_dataset['EIRRouteKey'].astype('string').str.strip()
properties_dataset['EIRUID'] = properties_dataset['EIRUID'].astype('string').str.strip()

In [116]:
#Remove non existant Routing Keys, aroun 1100
mask = ~properties_dataset['EIRRouteKey'].isin(eircodes_csv['RoutingKey'])
properties_dataset['EIRRouteKey'] = np.where(mask, np.nan, properties_dataset['EIRRouteKey'])



In [117]:
# Drop columns no longer needed
properties_dataset = properties_dataset.drop(columns=['Address','Eircode','Street','SNumber','EIRUID'],axis=1)

In [118]:
display(properties_dataset.describe(include='all'))
completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")


,Date,County,Price,Description,Size,Neighbourhood,Area,SRoad,EIRRouteKey
count,726373,726373,7.263730e+05,726373,52831,726373,643630,724970,189700
unique,NaN,26,NaN,5,6,56851,12925,130552,139
top,NaN,dublin,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,lucan,dublin,,v94
freq,NaN,227727,NaN,601601,38097,3476,34007,18346,6889
mean,2019-01-03 17:08:41.439535360,NaN,3.046492e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,2010-01-01 00:00:00,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,NaN
25%,2015-11-27 00:00:00,NaN,1.380000e+05,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-04-08 00:00:00,NaN,2.312775e+05,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-06-23 00:00:00,NaN,3.485000e+05,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-06-06 00:00:00,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,NaN


Completeness of EIRRouteKey: 26.12%


# Primer enriquecimiento usando los codigos de Ruta conocidos de eircodes.csv

In [119]:
properties_dataset = properties_dataset.merge(
    eircodes_csv[['PostTown', 'RoutingKey']],
    left_on='Neighbourhood',
    right_on='PostTown',
    how='left'
)
properties_dataset.rename(columns={'RoutingKey': 'RKey_Nei'} , inplace=True)

properties_dataset = properties_dataset.merge(
    eircodes_csv[['PostTown', 'RoutingKey']],
    left_on='Area',
    right_on='PostTown',
    how='left'
)
properties_dataset.rename(columns={'RoutingKey': 'RKey_Area'} , inplace=True)

properties_dataset = properties_dataset.merge(
    eircodes_csv[['PostTown', 'RoutingKey']],
    left_on='County',
    right_on='PostTown',
    how='left'
)
properties_dataset.rename(columns={'RoutingKey': 'RKey_Coun'} , inplace=True)



In [120]:
# If EIRRouteKey is empy replace with RKey_Nei, RKey_Area, RKey_Coun
properties_dataset['EIRRouteKey'] = (
    properties_dataset['EIRRouteKey']
    .fillna(properties_dataset['RKey_Nei'])
    .fillna(properties_dataset['RKey_Area'])
    .fillna(properties_dataset['RKey_Coun'])
)

completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")


Completeness of EIRRouteKey: 79.15%


In [121]:

# Drop columns no longer needed
properties_dataset = properties_dataset.drop(columns=['PostTown_x', 'RKey_Nei', 'PostTown_y', 'RKey_Area', 'PostTown', 'RKey_Coun'],axis=1)
display(properties_dataset.describe(include='all'))

,Date,County,Price,Description,Size,Neighbourhood,Area,SRoad,EIRRouteKey
count,726373,726373,7.263730e+05,726373,52831,726373,643630,724970,574920
unique,NaN,26,NaN,5,6,56851,12925,130552,139
top,NaN,dublin,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,lucan,dublin,,h91
freq,NaN,227727,NaN,601601,38097,3476,34007,18346,25499
mean,2019-01-03 17:08:41.439535360,NaN,3.046492e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,2010-01-01 00:00:00,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,NaN
25%,2015-11-27 00:00:00,NaN,1.380000e+05,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-04-08 00:00:00,NaN,2.312775e+05,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-06-23 00:00:00,NaN,3.485000e+05,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-06-06 00:00:00,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,NaN


# Segundo enriquesimiento usando los vecinos ya conocidos que tiene el codigo de Ruta

In [122]:
# Lets try using the now known neighbours to find properties sold in the same neighbourhood
neighbours_dataset = properties_dataset.loc[properties_dataset['EIRRouteKey'].notna(), ['EIRRouteKey', 'County', 'Neighbourhood', 'SRoad']].copy().reindex()
neighbours_dataset = neighbours_dataset.drop_duplicates(subset=['County', 'Neighbourhood', 'SRoad'])
neighbours_dataset.rename(columns={'EIRRouteKey': 'N_key'}, inplace=True)
neighbours_dataset.fillna('Not Available', inplace=True)

In [123]:
# With more accuracy using 3 matching fields
properties_dataset = properties_dataset.merge(
    neighbours_dataset[['County','Neighbourhood', 'SRoad', 'N_key']],
    left_on=['County','Neighbourhood', 'SRoad'],
    right_on=['County','Neighbourhood', 'SRoad'],
    how='left'
)

properties_dataset['EIRRouteKey'] = (
    properties_dataset['EIRRouteKey']
    .fillna(properties_dataset['N_key'])
)

# Drop columns no longer needed
properties_dataset = properties_dataset.drop(columns=['N_key'],axis=1)

display(properties_dataset.describe(include='all'))
completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")


,Date,County,Price,Description,Size,Neighbourhood,Area,SRoad,EIRRouteKey
count,726373,726373,7.263730e+05,726373,52831,726373,643630,724970,653846
unique,NaN,26,NaN,5,6,56851,12925,130552,139
top,NaN,dublin,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,lucan,dublin,,v94
freq,NaN,227727,NaN,601601,38097,3476,34007,18346,25720
mean,2019-01-03 17:08:41.439535360,NaN,3.046492e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,2010-01-01 00:00:00,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,NaN
25%,2015-11-27 00:00:00,NaN,1.380000e+05,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-04-08 00:00:00,NaN,2.312775e+05,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-06-23 00:00:00,NaN,3.485000e+05,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-06-06 00:00:00,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,NaN


Completeness of EIRRouteKey: 90.02%


In [124]:
# Lets try using the now known neighbours to find properties sold in the same neighbourhood
neighbours_dataset = properties_dataset.loc[properties_dataset['EIRRouteKey'].notna(), ['EIRRouteKey', 'County', 'Neighbourhood']].copy().reindex()
neighbours_dataset = neighbours_dataset.drop_duplicates(subset=['County', 'Neighbourhood'])
neighbours_dataset.rename(columns={'EIRRouteKey': 'N_key'}, inplace=True)
neighbours_dataset.fillna('Not Available', inplace=True)

# With more accuracy using 2 matching fields

properties_dataset = properties_dataset.merge(
    neighbours_dataset[['County','Neighbourhood', 'N_key']],
    left_on=['County','Neighbourhood'],
    right_on=['County','Neighbourhood'],
    how='left'
)


properties_dataset['EIRRouteKey'] = (
    properties_dataset['EIRRouteKey']
    .fillna(properties_dataset['N_key'])
)

# Drop columns no longer needed
properties_dataset = properties_dataset.drop(columns=['N_key'],axis=1)

display(properties_dataset.describe(include='all'))
completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")

,Date,County,Price,Description,Size,Neighbourhood,Area,SRoad,EIRRouteKey
count,726373,726373,7.263730e+05,726373,52831,726373,643630,724970,704460
unique,NaN,26,NaN,5,6,56851,12925,130552,139
top,NaN,dublin,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,lucan,dublin,,v94
freq,NaN,227727,NaN,601601,38097,3476,34007,18346,26475
mean,2019-01-03 17:08:41.439535360,NaN,3.046492e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,2010-01-01 00:00:00,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,NaN
25%,2015-11-27 00:00:00,NaN,1.380000e+05,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-04-08 00:00:00,NaN,2.312775e+05,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-06-23 00:00:00,NaN,3.485000e+05,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-06-06 00:00:00,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,NaN


Completeness of EIRRouteKey: 96.98%


In [125]:
# Lets try using the now known neighbours to find properties sold in the same neighbourhood
neighbours_dataset = properties_dataset.loc[properties_dataset['EIRRouteKey'].notna(), ['EIRRouteKey', 'County', 'SRoad']].copy().reindex()
neighbours_dataset = neighbours_dataset.drop_duplicates(subset=['County', 'SRoad'])
neighbours_dataset.rename(columns={'EIRRouteKey': 'N_key'}, inplace=True)
neighbours_dataset.fillna('Not Available', inplace=True)

# With more accuracy using 2 matching fields

properties_dataset = properties_dataset.merge(
    neighbours_dataset[['County','SRoad', 'N_key']],
    left_on=['County','SRoad'],
    right_on=['County','SRoad'],
    how='left'
)


properties_dataset['EIRRouteKey'] = (
    properties_dataset['EIRRouteKey']
    .fillna(properties_dataset['N_key'])
)

# Drop columns no longer needed
properties_dataset = properties_dataset.drop(columns=['N_key'],axis=1)

display(properties_dataset.describe(include='all'))
completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")

,Date,County,Price,Description,Size,Neighbourhood,Area,SRoad,EIRRouteKey
count,726373,726373,7.263730e+05,726373,52831,726373,643630,724970,717690
unique,NaN,26,NaN,5,6,56851,12925,130552,139
top,NaN,dublin,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,lucan,dublin,,v94
freq,NaN,227727,NaN,601601,38097,3476,34007,18346,26575
mean,2019-01-03 17:08:41.439535360,NaN,3.046492e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,2010-01-01 00:00:00,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,NaN
25%,2015-11-27 00:00:00,NaN,1.380000e+05,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-04-08 00:00:00,NaN,2.312775e+05,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-06-23 00:00:00,NaN,3.485000e+05,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-06-06 00:00:00,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,NaN


Completeness of EIRRouteKey: 98.80%


In [126]:
# Drop the remaining 1.8% with empty EIRRouteKey
properties_dataset = properties_dataset.dropna(subset=['EIRRouteKey'])

completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")

Completeness of EIRRouteKey: 100.00%


In [127]:
# Group by RoutingKey and join PostTown values with a comma
merged_eircodes = eircodes_csv.groupby('RoutingKey', as_index=False).agg({
    'PostTown': ', '.join,
    'PostCounty': 'first'  # or another aggregation if needed
})

display(merged_eircodes.describe(include='all'))

,RoutingKey,PostTown,PostCounty
count,139,139,139
unique,139,139,26
top,a41,ballyboughal,dublin
freq,1,1,34


In [ ]:
# Add the town and fill the area
properties_dataset = properties_dataset.merge(
    merged_eircodes[['PostTown', 'RoutingKey']],
    left_on='EIRRouteKey',
    right_on='RoutingKey',
    how='left'
)

In [130]:
properties_dataset.fillna({'Area':'Not Available'}, inplace=True)

In [131]:
#Drop This since in the EIR column already
if 'RoutingKey' in properties_dataset.columns:
    properties_dataset.drop(columns={'RoutingKey'}, axis=1, inplace=True)

In [84]:
display(properties_dataset.describe(include='all'))

,Date,County,Price,Description,Size,Neighbourhood,SRoad,EIRRouteKey,PostTown,RoutingKey
count,929560,929560,9.295600e+05,929560,70273,929560,928382,929560,929560,929560
unique,NaN,26,NaN,5,6,52273,125891,139,160,139
top,NaN,dublin,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,drogheda,,a92,limerick,a92
freq,NaN,255985,NaN,756021,51209,12276,21254,76640,26575,76640
mean,2019-01-20 15:40:26.758466560,NaN,3.076318e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2010-01-03 00:00:00,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2015-12-10 00:00:00,NaN,1.499500e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-05-03 00:00:00,NaN,2.420000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-07-08 00:00:00,NaN,3.550000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-06-06 00:00:00,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Reemplazar la columna Desctiption por un boolean

In [134]:
#  (New = True else = False)

def format_desc(desc):
    if isinstance(desc, str) and 'new' in desc.lower():
        return True
    else:
        return False

properties_dataset['NewBuilding'] = properties_dataset['Description'].apply(format_desc)


In [135]:
if 'Description' in properties_dataset.columns:
    properties_dataset.drop(columns={'Description'}, axis=1, inplace=True)

In [ ]:
# https://data.cso.ie/table/HPM04  ??? Quiza util?

# Auxiliares para testeo

In [ ]:
# Folium -- Library for grafics

# Copy the dataset for testing purposes

properties_dataset_test = properties_dataset.copy()

In [ ]:
# Testing formatting

display(properties_dataset_test[['SNumber', 'SRoad','Street', 'Neighbourhood', 'Area', 'Address', 'Eircode']].sample(10))


In [136]:
# Save the cleaned dataset to a new CSV file
output_csv = osp.join("res", "PPR-ALL-formated.csv")
properties_dataset.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"Dataset cleaned and saved to {output_csv}")

Dataset cleaned and saved to res\PPR-ALL-formated.csv


In [ ]:
# Load the cleaned dataset to a new CSV file
input_csv = osp.join("res", "PPR-ALL-formated.csv")
properties_dataset = pd.read_csv(input_csv, sep=',', header=0, engine='python', encoding='utf-8-sig')

In [39]:

display(properties_dataset.head(5))
display(properties_dataset.info()) 
display(properties_dataset.describe(include='all'))

,Date,Address,County,Eircode,Price,Description,Size,Street,Neighbourhood,Area,SNumber,SRoad,EIRRouteKey,EIRUID
0,NaT,"5 braemor drive, churchtown, co.dublin",dublin,<NA>,343000.0,second-hand dwelling house /apartment,<NA>,5 braemor drive,churchtown,dublin,5,braemor drive,<NA>,<NA>
1,NaT,"134 ashewood walk, summerhill lane, portlaoise",laois,<NA>,185000.0,new dwelling house /apartment,greater than or equal to 38 sq metres and less...,134 ashewood walk,summerhill lane,portlaoise,134,ashewood walk,R32,<NA>
2,NaT,"1 meadow avenue, dundrum, dublin 14",dublin,<NA>,438500.0,second-hand dwelling house /apartment,<NA>,1 meadow avenue,dundrum,dublin 14,1,meadow avenue,D14,<NA>
3,NaT,"1 the haven, mornington",meath,<NA>,400000.0,second-hand dwelling house /apartment,<NA>,1 the haven,mornington,<NA>,1,the haven,<NA>,<NA>
4,NaT,"11 melville heights, kilkenny",kilkenny,<NA>,160000.0,second-hand dwelling house /apartment,<NA>,11 melville heights,kilkenny,<NA>,11,melville heights,R95,<NA>


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726373 entries, 0 to 726372
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           0 non-null       datetime64[ns]
 1   Address        726373 non-null  string        
 2   County         726373 non-null  string        
 3   Eircode        189877 non-null  string        
 4   Price          726373 non-null  float64       
 5   Description    726373 non-null  string        
 6   Size           52831 non-null   string        
 7   Street         726373 non-null  string        
 8   Neighbourhood  726373 non-null  string        
 9   Area           643534 non-null  string        
 10  SNumber        726373 non-null  int64         
 11  SRoad          712987 non-null  string        
 12  EIRRouteKey    523911 non-null  string        
 13  EIRUID         189877 non-null  string        
dtypes: datetime64[ns](1), float64(1), int64(1), string(1

None

,Date,Address,County,Eircode,Price,Description,Size,Street,Neighbourhood,Area,SNumber,SRoad,EIRRouteKey,EIRUID
count,0,726373,726373,189877,7.263730e+05,726373,52831,726373,726373,643534,7.263730e+05,712987,523911,189877
unique,NaN,638149,26,182871,NaN,5,6,491632,56851,12923,NaN,137350,412,51206
top,NaN,"broomfield, midleton",dublin,d24w9nn,NaN,second-hand dwelling house /apartment,greater than or equal to 38 sq metres and less...,1,lucan,dublin,NaN,apartment,R51,w9nn
freq,NaN,22,227727,34,NaN,601601,38097,1685,3476,34007,NaN,5369,29446,34
mean,NaT,NaN,NaN,NaN,3.046492e+05,NaN,NaN,NaN,NaN,NaN,2.827779e+11,NaN,NaN,NaN
min,NaT,NaN,NaN,NaN,5.001000e+03,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN
25%,NaT,NaN,NaN,NaN,1.380000e+05,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN,NaN,NaN
50%,NaT,NaN,NaN,NaN,2.312775e+05,NaN,NaN,NaN,NaN,NaN,1.300000e+01,NaN,NaN,NaN
75%,NaT,NaN,NaN,NaN,3.485000e+05,NaN,NaN,NaN,NaN,NaN,3.700000e+01,NaN,NaN,NaN
max,NaT,NaN,NaN,NaN,3.876652e+08,NaN,NaN,NaN,NaN,NaN,2.054022e+17,NaN,NaN,NaN


## Estas son funciones viejas que utilize al comienzo para popular el codigo de ruta pero es muy lento


In [62]:
# Optional, not used!!

# Get missing EIRCODEs -- Access to the EIRCODE DB from the IRL gob costs 3000 EU
# Will be using the Eircode CSV file with the Routiong Key and the neighbourhoods to find the possible EIRRouteKey
# Select rows where Eircode is missing (nan or empty string)
# Takes around 4 minutes to run but it is worth it

missing_eircodes = properties_dataset[properties_dataset['EIRRouteKey'].isna()]


for index, row in missing_eircodes.iterrows():
    neighbourhood = row['Neighbourhood']
    area = row['Area']
    county = row['County']
    
    # Find matching EIRRouteKey in eircodes_csv matching the town which is the most accurate, the value muight be in the neighbourhood, area or county in this importance
    matching_eircodes = eircodes_csv[
        (eircodes_csv['PostTown'] == neighbourhood) | 
        (eircodes_csv['PostTown'] == area) | 
        (eircodes_csv['PostTown'] == county)
    ]
    
    if not matching_eircodes.empty:
        # If there are matches, assign the first one to the missing Eircode
        missing_eircodes.at[index, 'EIRRouteKey'] = matching_eircodes.iloc[0]['RoutingKey']
    else:
        # If no matches found, keep EIRRouteKey as NaN
        missing_eircodes.at[index, 'EIRRouteKey'] = np.nan

# Update the original DataFrame with the missing Eircodes
properties_dataset.update(missing_eircodes[['EIRRouteKey']])
# Went from 26% missing EIRRouteKey to 72% completeness for EIRRouteKey !!

completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")

Completeness of EIRRouteKey: 90.91%


In [ ]:
# Optional, not used!!
# 202k missing * 268K unique options * 3 combinations if -- This will take a while.... 291Min

cont = 0

for index, row in missing_eircodes.iterrows():
    SRoad = row['SRoad']
    neighbourhood = row['Neighbourhood']
    area = row['Area']
    county = row['County']
    cont += 1
    if cont % 10000 == 0:
        print("Procesado:", cont)

    # Find matching combinations of County with the other elements since they wont repeat by county if they match.
    matching_eircodes = neighbours_dataset[
        ((neighbours_dataset['County'] == county) & (neighbours_dataset['SRoad'] == SRoad)) |
        ((neighbours_dataset['County'] == county) & (neighbours_dataset['Neighbourhood'] == SRoad)) |
        ((neighbours_dataset['County'] == county) & (neighbours_dataset['Area'] == SRoad))
    ]
    
    if not matching_eircodes.empty:
        # If there are matches, assign the first one to the missing Eircode
        missing_eircodes.at[index, 'EIRRouteKey'] = matching_eircodes.iloc[0]['EIRRouteKey']
    else:
        # If no matches found, keep EIRRouteKey as NaN
        missing_eircodes.at[index, 'EIRRouteKey'] = np.nan
    

# Update the original DataFrame with the missing Eircodes
properties_dataset.update(missing_eircodes[['EIRRouteKey']])

completeness = properties_dataset['EIRRouteKey'].notna().mean() * 100
print(f"Completeness of EIRRouteKey: {completeness:.2f}%")

# Save the cleaned dataset to a new CSV file
output_csv = os.path.join("res", "PPR-ALL-cleaned2.csv")
properties_dataset.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"Dataset cleaned and saved to {output_csv}")

Procesado: 10000
Procesado: 20000
Procesado: 30000
Procesado: 40000
Procesado: 50000
Procesado: 60000
Procesado: 70000
Procesado: 80000
Procesado: 90000
Procesado: 100000
Procesado: 110000
Procesado: 120000
Procesado: 130000
Procesado: 140000
Procesado: 150000
Procesado: 160000
Procesado: 170000
Procesado: 180000
Procesado: 190000
Procesado: 200000
Completeness of EIRRouteKey: 91.92%
Dataset cleaned and saved to res\PPR-ALL-cleaned2.csv
